# **Homework 1: Linear Regression**

# **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [1]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('./../data/train.csv', encoding = 'big5')

# **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [2]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

# **Extract Features (1)**
![圖片說明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![圖片說明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [3]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [4]:
items = np.empty([18, 480*12])
for i in range(18):
    for m in range(12):
        if m == 0:
            temp = month_data[m][i,:]
        else:
            temp = np.concatenate((temp, month_data[m][i,:]), axis = -1)
    items[i] = temp

In [5]:
cor = np.corrcoef(items)[9]
cor

array([-0.01712724,  0.25465706,  0.28311942,  0.29177826,  0.02997038,
        0.44911349,  0.37556381,  0.35667002,  0.77642643,  1.        ,
       -0.06265388, -0.26419607,  0.3708308 ,  0.3521594 ,  0.18613794,
        0.15699025, -0.08470312, -0.04545785])

In [6]:
items_bool = np.copy(cor)
items_bool[items_bool > 0.3] = 1
items_bool[items_bool <= 0.3] = 0
items_bool

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
       0.])

In [7]:
new_month_data = {}
for month in range(12):
    j = 0
    sample = np.empty([int(np.sum(items_bool)), 480])
    for i in range(18):
        if items_bool[i] == 1:
            sample[j,:] = month_data[month][i,:]
            j += 1
    new_month_data[month] = sample

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [8]:
x = np.empty([12 * 471, int(np.sum(items_bool)) * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = new_month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

[[16.   9.2  8.2 ...  1.9  1.9  2.1]
 [ 9.2  8.2  6.9 ...  1.9  2.1  2. ]
 [ 8.2  6.9  6.8 ...  2.1  2.   2. ]
 ...
 [ 6.4  5.7  6.1 ...  2.   1.9  1.9]
 [ 5.7  6.1  8.  ...  1.9  1.9  1.9]
 [ 6.1  8.  11.  ...  1.9  1.9  2. ]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


# **Normalize (1)**


In [9]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[ 0.94869554, -0.14813488, -0.31066014, ...,  0.32976505,
         0.33039252,  1.42801429],
       [-0.14778078, -0.30935073, -0.52008895, ...,  0.32976505,
         1.42627194,  0.87988955],
       [-0.30902729, -0.51893132, -0.53619886, ...,  1.42549229,
         0.87833223,  0.87988955],
       ...,
       [-0.59927103, -0.71239034, -0.64896822, ...,  0.87762867,
         0.33039252,  0.33176481],
       [-0.71214359, -0.647904  , -0.34287996, ...,  0.32976505,
         0.33039252,  0.33176481],
       [-0.64764498, -0.3415939 ,  0.14041729, ...,  0.32976505,
         0.33039252,  0.87988955]])

#**Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中 report 的第二題、第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [10]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[ 0.94869554 -0.14813488 -0.31066014 ...  0.32976505  0.33039252
   1.42801429]
 [-0.14778078 -0.30935073 -0.52008895 ...  0.32976505  1.42627194
   0.87988955]
 [-0.30902729 -0.51893132 -0.53619886 ...  1.42549229  0.87833223
   0.87988955]
 ...
 [-0.97013801 -0.56729608 -0.06901152 ... -0.7659622  -0.7654869
  -0.21635992]
 [-0.56702172 -0.06752696  0.14041729 ... -0.7659622  -0.21754719
  -0.21635992]
 [-0.06715752  0.14205364  0.94591271 ... -0.21809858 -0.21754719
  -0.21635992]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.14246295  0.94813285 -0.68118803 ... -0.21809858 -0.21754719
  -0.21635992]
 [ 0.94869554 -0.68014717 -1.30947446 ... -0.21809858 -0.21754719
  -0.76448466]
 [-0.67989428 -1.30888896 -1.27725464 ... -0.21809858 -0.7654869
  -0.76448466]
 ...
 [-0.59927103 -0.71239034 -0.64896822 ...  0.87762867  0.33039252
   0.33176481]
 [-0.71214359 -0.647904   -0.34287996 ...  0.32976505  0.33039252
   0.33176481]
 [-0.64764498 -0.3415939   0.14041729 ...  0.3297650

# **Training**
![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [ ]:
dim = int(np.sum(items_bool) * 9 + 1)
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 1.5
iter_time = 20000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# np.save('./../model/weight.npy', w)
w

# **Testing**
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [ ]:
testdata = pd.read_csv('./../data/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, int(np.sum(items_bool)*9)], dtype = float)
k = 0
for i in range(18):
    if items_bool[i] == 1:
        for j in range(240):
                test_x[j, k*9 : (k+1)*9] = test_data[18 * j + i, :]
        k += 1
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

# **Prediction**
說明圖同上

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

有了 weight 和測試資料即可預測 target。

In [ ]:
# w = np.load('./../model/weight.npy')
ans_y = np.dot(test_x, w)
ans_y

# **Save Prediction to CSV File**


In [ ]:
import csv
with open('./../results/submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

---
# Homework1 report 

## Problem1

### 使用四種不同的 learning rate 進行 training (其他參數需一致)，作圖並討論其收斂過程（橫軸為 iteration 次數，縱軸為 loss 的大小，四種 learning rate 的收斂線請以不同顏色呈現在一張圖裡做比較）。

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = np.empty([12 * 471, int(np.sum(items_bool)) * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = new_month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
# Normalize
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [ ]:
learning_rate_list = [10**power for power in range(-1,3)]
loss_history = []
dim = int(np.sum(items_bool) * 9 + 1)
iter_time = 1000
eps = 0.0000000001
for lr in learning_rate_list:
    print("Use learning rate:", lr)
    history = []
    w = np.zeros([dim, 1])
    temp_x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
    learning_rate = lr
    adagrad = np.zeros([dim, 1])
    for t in range(iter_time):
        loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y, 2))/471/12)#rmse
        history.append(loss)
        if(t%100==0):
            print(str(t) + ":" + str(loss))
        gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y) #dim*1
        adagrad += gradient ** 2
        w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    loss_history.append(history)
loss_history = np.array(loss_history)

In [ ]:
for i in range(loss_history.shape[0]):
    plt.plot(
        np.arange(loss_history.shape[1])+1,
        np.log10(loss_history[i,:]),
        label = 'learning rate: ' + str(learning_rate_list[i])
    )
plt.legend()
plt.title('loss $(log_{10})$')
plt.xlabel('iteration')
plt.ylabel('log loss')

### You will find that the best learning rate $\eta$ should be between 10 and 100
### Let's try to tune the learning rate

In [ ]:
learning_rate_list = np.arange(10,110,10)
loss_history = []
dim = int(np.sum(items_bool) * 9 + 1)
iter_time = 1000
eps = 0.0000000001
for lr in learning_rate_list:
    print("Use learning rate:", lr)
    history = []
    w = np.zeros([dim, 1])
    temp_x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
    learning_rate = lr
    adagrad = np.zeros([dim, 1])
    for t in range(iter_time):
        loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y, 2))/471/12)#rmse
        history.append(loss)
        if(t%100==0):
            print(str(t) + ":" + str(loss))
        gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y) #dim*1
        adagrad += gradient ** 2
        w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    loss_history.append(history)
loss_history = np.array(loss_history)

In [ ]:
for i in range(loss_history.shape[0]):
    plt.plot(
        (np.arange(loss_history.shape[1])+1)[750:],
        (np.log10(loss_history[i,:]))[750:],
        label = 'learning rate: ' + str(learning_rate_list[i])
    )
plt.legend()
plt.title('loss $(log_{10})$')
plt.xlabel('iteration')
plt.ylabel('log loss')

### 5 to 15

In [ ]:
learning_rate_list = np.arange(5,16,1)
loss_history = []
dim = int(np.sum(items_bool) * 9 + 1)
iter_time = 1000
eps = 0.0000000001
for lr in learning_rate_list:
    print("Use learning rate:", lr)
    history = []
    w = np.zeros([dim, 1])
    temp_x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
    learning_rate = lr
    adagrad = np.zeros([dim, 1])
    for t in range(iter_time):
        loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y, 2))/471/12)#rmse
        history.append(loss)
        if(t%100==0):
            print(str(t) + ":" + str(loss))
        gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y) #dim*1
        adagrad += gradient ** 2
        w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    loss_history.append(history)
loss_history = np.array(loss_history)

In [ ]:
for i in range(loss_history.shape[0]):
    plt.plot(
        (np.arange(loss_history.shape[1])+1)[1750:],
        (np.log10(loss_history[i,:]))[1750:],
        label = 'learning rate: ' + str(learning_rate_list[i])
    )
plt.legend()
plt.title('loss $(log_{10})$')
plt.xlabel('iteration')
plt.ylabel('log loss')

In [ ]:
loss_history[:,-1], loss_history[-4,-1], learning_rate_list[-4]

### 1 to 5

In [ ]:
learning_rate_list = np.arange(1,5.5,0.5)
loss_history = []
dim = int(np.sum(items_bool) * 9 + 1)
iter_time = 2000
eps = 0.0000000001
for lr in learning_rate_list:
    print("Use learning rate:", lr)
    history = []
    w = np.zeros([dim, 1])
    temp_x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
    learning_rate = lr
    adagrad = np.zeros([dim, 1])
    for t in range(iter_time):
        loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y, 2))/471/12)#rmse
        history.append(loss)
        if(t%100==0):
            print(str(t) + ":" + str(loss))
        gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y) #dim*1
        adagrad += gradient ** 2
        w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    loss_history.append(history)
loss_history = np.array(loss_history)

In [ ]:
for i in range(loss_history.shape[0]):
    plt.plot(
        (np.arange(loss_history.shape[1])+1)[1900:],
        (np.log10(loss_history[i,:]))[1900:],
        label = 'learning rate: ' + str(learning_rate_list[i])
    )
plt.legend()
plt.title('loss $(log_{10})$')
plt.xlabel('iteration')
plt.ylabel('log loss')

In [ ]:
loss_history[:,-1]
# , loss_history[-4,-1], learning_rate_list[-4]

### Training

In [ ]:
dim = int(np.sum(items_bool) * 9 + 1)
w = np.zeros([dim, 1])
temp_x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 1
iter_time = 20000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('./../model/weight3.npy', w)
w

### Predict

In [ ]:
w = np.load('./../model/weight3.npy')
ans_y = np.dot(test_x, w)
ans_y

### Save

In [ ]:
import csv
with open('./../results/submit3.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)